In [1]:
# Import basic libraries
from palmerpenguins import load_penguins
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import time
from sklearn.preprocessing import LabelEncoder
import os
os.chdir("/Users/hoangthuyduongvu/Desktop/FuzzSIM")


In [2]:
# Import personalized libraries
from fuzz.src.capacity import *
from fuzz.choquet.choquet import *
from fuzz.src.norm import *
from fuzz.src.knn import KNNFuzz
from fuzz.src.sim import S1, S2, S3
from fuzz.optim import *
from fuzz.utils import *
from fuzz.eval import leave_one_out
from fuzz.dataloader import *
from fuzz.choquet.d_choquet import *

In [3]:
features = [0, 1, 2, 3]

mobius = generate_mobius(features, 2)
capacity = mobius_to_capacity(mobius, features)

print("Möbius:")
for i in range(len(mobius)):
    print(f"Mobius of {mobius[i].X} is {mobius[i].mu:.3f}")

print("\nDerived Capacity:")
for c in capacity:
    print(f"Capacity of {c.X} is {c.mu:.3f}")


Möbius:
Mobius of [] is 0.000
Mobius of [0] is 0.664
Mobius of [1] is 0.836
Mobius of [2] is 0.839
Mobius of [3] is 0.658
Mobius of [0, 1] is 0.750
Mobius of [0, 2] is 0.576
Mobius of [0, 3] is 0.370
Mobius of [1, 2] is 0.530
Mobius of [1, 3] is 0.812
Mobius of [2, 3] is 0.564

Derived Capacity:
Capacity of [] is 0.000
Capacity of [0] is 0.101
Capacity of [1] is 0.127
Capacity of [2] is 0.127
Capacity of [3] is 0.100
Capacity of [0, 1] is 0.341
Capacity of [0, 2] is 0.315
Capacity of [0, 3] is 0.256
Capacity of [1, 2] is 0.334
Capacity of [1, 3] is 0.350
Capacity of [2, 3] is 0.312
Capacity of [0, 1, 2] is 0.636
Capacity of [0, 1, 3] is 0.620
Capacity of [0, 2, 3] is 0.556
Capacity of [1, 2, 3] is 0.642
Capacity of [0, 1, 2, 3] is 1.000


In [4]:
for subset in powerset(features):
    print(f"Subset: {set(subset)}")

Subset: set()
Subset: {0}
Subset: {1}
Subset: {2}
Subset: {3}
Subset: {0, 1}
Subset: {0, 2}
Subset: {0, 3}
Subset: {1, 2}
Subset: {1, 3}
Subset: {2, 3}
Subset: {0, 1, 2}
Subset: {0, 1, 3}
Subset: {0, 2, 3}
Subset: {1, 2, 3}
Subset: {0, 1, 2, 3}


In [5]:
X = np.array([0.2, 0.5, 0.8])
choquet_value = Choquet_classic(X, capacity, verbose=True)
print("Choquet integral:", choquet_value)


val_check: [0, 1, 2] - capacity_observation_i: 0.6356243879535027 - val_check2: [1, 2] - capacity_observation_i_1: 0.3340071001121248
val_check: [1, 2] - capacity_observation_i: 0.3340071001121248 - val_check2: [2] - capacity_observation_i_1: 0.1270750441169277
val_check: [2] - capacity_observation_i: 0.1270750441169277 - val_check2: [] - capacity_observation_i_1: 0.0
Choquet integral: 0.2654495208594163


In [6]:
iris_data = load_iris()
iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris['target'] = iris_data.target
data, labels = iris.iloc[:, :-1].values, iris.iloc[:, -1].values
data = batch_norm(np.array(data, dtype=np.float32))

In [7]:
choquet = Choquet(X=data[0], mu=capacity, version='classic', p=1, q=1)
print(choquet.choquet)
# choquet.Choquet_classic(verbose=True)

0.6216421453387129


In [8]:
X = data[0]
choquet_value = Choquet_classic(X, capacity, verbose=True)
print("Choquet integral:", choquet_value)


val_check: [3, 2, 1, 0] - capacity_observation_i: 1.0 - val_check2: [2, 1, 0] - capacity_observation_i_1: 0.6356243879535027
val_check: [2, 1, 0] - capacity_observation_i: 0.6356243879535027 - val_check2: [1, 0] - capacity_observation_i_1: 0.34104358871022816
val_check: [1, 0] - capacity_observation_i: 0.34104358871022816 - val_check2: [0] - capacity_observation_i_1: 0.10063573564552829
val_check: [0] - capacity_observation_i: 0.10063573564552829 - val_check2: [] - capacity_observation_i_1: 0.0
Choquet integral: 0.6216421453387129


In [9]:
cross = crossover(
    parent1=generate_capacity(enumerate_permute_unit(data[0])), 
    parent2=generate_capacity(enumerate_permute_unit(data[0]))
)
print("\nCrossed Capacity:")
for c in cross:
    print(f"Capacity of {c.X} is {c.mu:.3f}")


Crossed Capacity:
Capacity of [] is 0.000
Capacity of [3] is 0.053
Capacity of [2] is 0.215
Capacity of [1] is 0.182
Capacity of [0] is 0.190
Capacity of [3, 2] is 0.225
Capacity of [3, 1] is 0.256
Capacity of [3, 0] is 0.277
Capacity of [2, 1] is 0.667
Capacity of [2, 0] is 0.616
Capacity of [1, 0] is 0.641
Capacity of [3, 2, 1] is 0.675
Capacity of [3, 2, 0] is 0.772
Capacity of [3, 1, 0] is 0.782
Capacity of [2, 1, 0] is 0.785
Capacity of [3, 2, 1, 0] is 1.000


In [10]:
mutated_mobius = mutate(capacity, mutation_rate=0.2)
print("\nMutated Möbius:")
for i in range(len(mutated_mobius)):
    print(f"Mobius of {mutated_mobius[i].X} is {mutated_mobius[i].mu:.3f}")


Mutated Möbius:
Mobius of [] is 0.000
Mobius of [0] is 0.101
Mobius of [1] is 0.127
Mobius of [2] is 0.127
Mobius of [3] is 0.100
Mobius of [0, 1] is 0.341
Mobius of [0, 2] is 0.315
Mobius of [0, 3] is 0.256
Mobius of [1, 2] is 0.334
Mobius of [1, 3] is 0.350
Mobius of [2, 3] is 0.312
Mobius of [0, 1, 2] is 0.554
Mobius of [0, 1, 3] is 0.651
Mobius of [0, 2, 3] is 0.554
Mobius of [1, 2, 3] is 0.642
Mobius of [0, 1, 2, 3] is 1.000


In [11]:
mutated_mu = mutate(capacity, mutation_rate=0.2)
print("\nMutated Möbius:")
for i in range(len(mutated_mu)):
    print(f"Mobius of {mutated_mu[i].X} is {mutated_mu[i].mu:.3f}")


Mutated Möbius:
Mobius of [] is 0.000
Mobius of [0] is 0.101
Mobius of [1] is 0.127
Mobius of [2] is 0.127
Mobius of [3] is 0.100
Mobius of [0, 1] is 0.341
Mobius of [0, 2] is 0.315
Mobius of [0, 3] is 0.256
Mobius of [1, 2] is 0.334
Mobius of [1, 3] is 0.338
Mobius of [2, 3] is 0.312
Mobius of [0, 1, 2] is 0.636
Mobius of [0, 1, 3] is 0.620
Mobius of [0, 2, 3] is 0.556
Mobius of [1, 2, 3] is 0.642
Mobius of [0, 1, 2, 3] is 1.000


In [ ]:
# LOO evaluation
# Define list for each sim level
l1 = []
l2 = []
l3 = []
k = 10

for k in range(1, k+1): 
    tic = time.time()
    mobius = generate_mobius(features)
    mu = mobius_to_capacity(mobius, features)

    print(f"Leave one out avec k = {k}")
    
    # SimLevel1
    res = leave_one_out(
            C = KNNFuzz(input_dimension=data.shape[1], mu=mu, k=3, sim=S1, choquet_version='d_choquet', p=1, q=1), 
            DS = (data, labels)
        )
    l1.append(res)
    print(f"SimLevel1: {res}")
    # Sim level 2
    res = leave_one_out(
            C = KNNFuzz(input_dimension=data.shape[1],mu=mu, k=k, sim=S2, choquet_version='d_choquet', p=1, q=1), 
            DS = (data, labels)
        )
    l2.append(res)
    print(f"SimLevel2: {res}")

    # Sim level 3
    res = leave_one_out(
            C = KNNFuzz(input_dimension=data.shape[1],mu=mu, k=k, sim=S3, choquet_version='d_choquet', p=1, q=1), 
            DS = (data, labels)
        )
    l3.append(res)
    print(f"SimLevel3: {res}\n")

    toc = time.time()
    print(f"Result in {(toc-tic):0.4f} seconds.")

# Plot results
plt.plot(l1, label="Sim level 1", marker='o')
plt.plot(l2, label="Sim level 2", marker='o')
plt.plot(l3, label="Sim level 3", marker='o')

# Config additional params
plt.ylabel("LOO accuracy")
plt.xlabel("k")
plt.title("Leave-One-Out Evaluation Results")
plt.grid()
plt.legend()

Leave one out avec k = 1
SimLevel1: 0.6666666666666666
SimLevel2: 0.44
SimLevel3: 0.9466666666666667

Result in 8.7991 seconds.
Leave one out avec k = 2
SimLevel1: 0.66
SimLevel2: 0.41333333333333333
SimLevel3: 0.9333333333333333

Result in 5.6880 seconds.
Leave one out avec k = 3
SimLevel1: 0.6666666666666666
SimLevel2: 0.28
SimLevel3: 0.9266666666666666

Result in 5.1779 seconds.
Leave one out avec k = 4
SimLevel1: 0.6666666666666666
SimLevel2: 0.18666666666666668
